In [1]:
import pprint
import torch
import numpy as np
import urllib
from PIL import Image

import timm
from timm.data import resolve_data_config
from timm.data.transforms_factory import create_transform


from torch import nn
import torchvision
import torchvision.models as models
import torchvision.transforms as T
from torchvision.datasets import CIFAR10
from torch.utils.data import DataLoader

from pl_bolts.models.self_supervised import SimCLR
from pytorch_lightning.core.lightning import LightningModule
from pytorch_lightning import Trainer
from pl_bolts.datasets import DummyDataset



## Creating Datasets


### CIFAR 10

In [ ]:
train_set = CIFAR10('./', download=True,
transform=T.Compose([
T.ToTensor(),
]), train=True)


test_set = CIFAR10('./', download=True,
transform=T.Compose([
T.ToTensor(),
]), train=False)

In [ ]:
batch_size = 64
trainloader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=True)
valloader = torch.utils.data.DataLoader(test_set, batch_size=batch_size, shuffle=True)

In [ ]:
dataiter = iter(trainloader)

images, labels = dataiter.next()

print(images.shape)
print(labels.shape)

### Dummy Dataset

In [2]:
ds = DummyDataset((3, 224, 224), (1,), num_samples=100)
dl = DataLoader(ds, batch_size=10)

## Model

The aim is to run the model in evaluation mode. For now, implement a single image format where we have a single image and a set of transformations and we calculate the mean embeddings in the end.

In [3]:
# class Encoder(LightningModule):
#     def __init__(self, encoder='resnet50_supervised'):
#         super().__init__()
        
#         self.encoder = encoder

#         #TODO: List of encoders in the configuration file
#         if encoder not in ['resnet50_supervised', 'simclr_r50', 'vit_base_patch8_224', 'vit_base_patch16_224_in21k', 'vit_base_patch32_224_in21k']:
#             raise AssertionError("Encoder not in the list of supported encoders.")
        
        
#         if(self.encoder == 'resnet50_supervised'):
#             backbone = models.resnet50(pretrained=True)
#             num_filters = backbone.fc.in_features
#             layers = list(backbone.children())[:-1]
#             self.feature_extractor = nn.Sequential(*layers)

#         elif(self.encoder == 'simclr_r50'):
#             weight_path = 'https://pl-bolts-weights.s3.us-east-2.amazonaws.com/simclr/bolts_simclr_imagenet/simclr_imagenet.ckpt'
#             simclr = SimCLR.load_from_checkpoint(weight_path, strict=False)
#             self.feature_extractor = simclr.encoder

#         elif(self.encoder == 'vit_base_patch16_224_in21k'):
#             self.feature_extractor = timm.create_model('vit_base_patch16_224_in21k', pretrained=True, num_classes=0)
#             config = resolve_data_config({}, model=self.feature_extractor)
#             transform = create_transform(**config)

#         elif(self.encoder == 'vit_base_patch32_224_in21k'):
#             self.feature_extractor = timm.create_model('vit_base_patch32_224_in21k', pretrained=True, num_classes=0)
#             config = resolve_data_config({}, model=self.feature_extractor)
#             transform = create_transform(**config)
            

#     def forward(self, x):
#         self.feature_extractor.eval()
#         with torch.no_grad():
#             if(self.encoder == 'resnet50_supervised'):
#                 representations = self.feature_extractor(x).flatten(1)
#             elif(self.encoder == 'simclr_r50'):
#                 representations = self.feature_extractor(x)[0]
#             elif(self.encoder == 'vit_base_patch16_224_in21k' or self.encoder == 'vit_base_patch32_224_in21k'):
#                 representations = self.feature_extractor(x)
            

#         return representations

In [25]:
class Encoder(LightningModule):
    def __init__(self, encoder='resnet50_supervised'):
        super().__init__()
        
        self.encoder = encoder

        #TODO: List of encoders in the configuration file
        if encoder not in ['resnet50_supervised', 'simclr_r50', 'vit_base_patch16_224_in21k', 'vit_base_patch32_224_in21k']:
            raise AssertionError("Encoder not in the list of supported encoders.")
        
        
        if(self.encoder == 'resnet50_supervised'):
            backbone = models.resnet50(pretrained=True)
            num_filters = backbone.fc.in_features
            layers = list(backbone.children())[:-1]
            self.feature_extractor = nn.Sequential(*layers)


        elif(self.encoder == 'simclr_r50'):
            weight_path = 'https://pl-bolts-weights.s3.us-east-2.amazonaws.com/simclr/bolts_simclr_imagenet/simclr_imagenet.ckpt'
            simclr = SimCLR.load_from_checkpoint(weight_path, strict=False)
            self.feature_extractor = simclr.encoder

        elif(self.encoder == 'vit_base_patch16_224_in21k'):
            self.feature_extractor = timm.create_model('vit_base_patch16_224_in21k', pretrained=True, num_classes=0)
            config = resolve_data_config({}, model=self.feature_extractor)
            transform = create_transform(**config)

        elif(self.encoder == 'vit_base_patch32_224_in21k'):
            self.feature_extractor = timm.create_model('vit_base_patch32_224_in21k', pretrained=True, num_classes=0)
            config = resolve_data_config({}, model=self.feature_extractor)
            transform = create_transform(**config)
            

    def forward(self, x):
        self.feature_extractor.eval()
        #batch_size, channels, height, width = x.size()

        with torch.no_grad():
            if(self.encoder == 'resnet50_supervised'):
                representations = self.feature_extractor(x).flatten(1)
            elif(self.encoder == 'simclr_r50'):
                representations = self.feature_extractor(x)[0]
            elif(self.encoder == 'vit_base_patch16_224_in21k' or self.encoder == 'vit_base_patch32_224_in21k'):
                representations = self.feature_extractor(x)
            

        return representations

In [24]:
aug_list = [T.RandomGrayscale(p=0.2), T.RandomHorizontalFlip(),
             T.ColorJitter(0.4, 0.4, 0.4, 0.1)]                     #This list should be in a configuration file


test_img = torch.randn(3,224,224)
test_tensor = torch.unsqueeze(test_img, 0)
print(test_tensor.shape)

torch.Size([1, 3, 224, 224])


In [ ]:
'''
For each image - 
1. Generate an augmented version using the augmentation from the list
2. Pass this augmented version through the encoder
3. Get the embedding and append it to a list
4. Get the mean embeddings
'''

# all_embeddings = torch.tensor([])
# encoder = Encoder(encoder='vit_base_patch32_224_in21k')

# for aug in aug_list:
#     print("Augmentation: ", aug)
#     preprocess = T.Compose([T.ToPILImage(), aug, T.ToTensor()])
#     aug_img = preprocess(test_img)
#     print(aug_img.size())

#     embedding = encoder(aug_img.unsqueeze(0))
#     print('embedding: ', embedding.size())

#     all_embeddings = torch.cat((all_embeddings,
#                                 embedding), 0)

# torch.mean(all_embeddings, 0).size()

In [27]:
'''
* Feed a dataset of images and obtain embeddings for all images. 
* Final output for a dataset should have size [num_samples, embedding_size]
* 'embedding_size' depends on the encoder used. [ResNet, SimCLR = 2048, ViTs = 768]
'''

all_embeddings = torch.tensor([])
final_dataset_embeddings = torch.tensor([])
encoder = Encoder(encoder='vit_base_patch32_224_in21k')

index=0
for x, y in dl:
    
    #print(" ############ BATCH ############")

    #For each image, apply each augmentation
    for i in range(x.size()[0]):
        all_embed_img = torch.tensor([])    #A tensor to hold embeddings of all augmentations of an image
        #print('Image: ', i+1)

        for aug in aug_list:
            #print("Augmentation: ", aug)
            preprocess = T.Compose([T.ToPILImage(), aug, T.ToTensor()])
            aug_img = preprocess(x[i])
            embedding = encoder(aug_img.unsqueeze(0))

            all_embed_img = torch.cat((all_embed_img,
                                    embedding), 0)

        all_embeddings = torch.cat((all_embeddings,
                                    torch.mean(all_embed_img, 0)), 0)

        #print('\n')

final_dataset_embeddings = torch.reshape(all_embeddings, (ds.num_samples, embedding.size()[1]))

final_dataset_embeddings.size() #Final size should be [num_samples, embedding_size]

/home/raman/miniconda3/envs/pt2/lib/python3.9/site-packages/torchvision/transforms/transforms.py:287: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  warnings.warn(


torch.Size([100, 768])